In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import altair as alt
import matplotlib.pyplot as plt
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
from utils.util import read_file, query_sequence, df_to_fasta, fasta_to_df

tqdm.pandas()

## Read Data 

In [54]:
#read benbow data 
train_data_path = "dataset/prev_studies/Treasure_Island/Benbow/"
fasta_file = os.path.join(train_data_path,"benbow_pos.fasta")
benbow_pos = read_file(fasta_file, '1')
fasta_file = os.path.join(train_data_path,"benbow_neg.fasta")
benbow_neg = read_file(fasta_file, '0')
benbow_data = pd.concat([benbow_pos, benbow_neg])

#read literature data 
literature_data = pd.read_excel("dataset/prev_studies/Treasure_Island/reference_data/GI_literature_set_table.xlsx")
literature_data = literature_data.assign(Label='1')

#get negative samples for literature data set 
literature_data_neg = benbow_data[(benbow_data['Accession'].isin(literature_data.Accession.unique())) & (benbow_data['Label'] == '0')]

#merge literature positive and negative samples
literature_data = pd.concat([literature_data[['Accession','Label','Start','End']], literature_data_neg])

#read IslandPick data
islandpick_data = pd.read_excel("dataset/prev_studies/IslandPick/RGP104.xlsx")

#list organisms covered in each data set
benbow_id = set(benbow_data['Accession'].values) 
islandpick_id = set(islandpick_data['Accession'].values)
literature_id = set(literature_data['Accession'].values)

## Query sequences from reference database and generate FASTA files

In [55]:
## query sequence from database from the given information, namely Accession, Start, and End, in each data 
## even though pandas offers parallel_apply function, it can't be performed in this function because of the multiple requests to the query database with the same email account
## one could modify this function but it will be left as it is

# benbow_data[['Sequence','Description']] = benbow_data.progress_apply(lambda x: query_sequence(x['Accession'], 
#                                                                                                                 x['Start'], 
#                                                                                                                 x['End']), 
#                                                                                         axis=1).to_list()


## in case of failure when querying the database, it will show "failed" in the Sequence column
## benbow_data[benbow_data['Sequence'].str.contains("failed")]

## or use the function df_to_fasta to write the above resulting data that now contains sequence and description into fasta file
## df_to_fasta function also has a parameter that can directly query and write the sequences into fasta file as the example below

# param = {'write_file':True,'filename':"dataset/train_data/benbow_data.fasta"} #whether write sequences to a fasta file specified in filename
# fasta_data = benbow_data.progress_apply(lambda x: df_to_fasta(x,
#                                                                        dna_only=True, #dna_only determines whether or not to process sequences with IUPAC codes
#                                                                        query_db=True, #query sequence from database
#                                                                        **param), axis=1).to_list()

## variable treasure_island_data can be replaced be any other data (benbow, islandpick, ...) declared above


In [56]:
## if the query_sequence function happens to be interrupted and there already exist a fasta file
## one can resume the writing by following steps below

## read the interrupted fasta file ("dataset/train_data/benbow_data.fasta")
# benbow_temp = fasta_to_df("path/to/the/interrupted_file", dna_only=True)

## find the remaining organisms having no sequence yet using outer join
# remaining_benbow = pd.merge(benbow_data, benbow_temp, on=['Accession','Start','End'], how="outer", indicator=True
#               ).query('_merge=="left_only"')[['Accession','Label_x','Start','End','Length']]
# remaining_benbow = remaining_benbow.rename(columns={'Label_x':'Label'})

## write (append) the remaining sequences into the written fasta file (make sure there is no redundancy)
# param = {'write_file':True,'filename':"path/to/the/interrupted_file"}                        
# remaining_benbow.progress_apply(lambda x: df_to_fasta(x,
#                                           dna_only=True,
#                                           query_db=True,
#                                           **param),
#                         axis=1)

### output: FASTA files named dataset/train_data/benbow_data.fasta, dataset/train_data/islandpick_data.fasta, dataset/test_data/literature_data

## Dataset from GI_Cluster (updated from ZislandExplorer)

In [57]:
#ZislandExplorer: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429010/
#GI_Cluster: https://doi.org/10.1142/S0219720018400103 https://github.com/icelu/GI_Cluster

#read dataset from GI_Cluster
file_path = "dataset/prev_studies/GI_Cluster/evaluation/C-dataset"
files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

data = []

for filename in files:
    
    with open(join(file_path,filename)) as file:
        for line in file:
            info = line.rstrip()
            start = info.split('\t')[0]
            end = info.split('\t')[1]

            if filename.split('.')[-1] == 'neg':
                label = '0'
            else:
                label = '1'
                
            data.append([filename.split('.')[0], int(start), int(end), label])

data_df = pd.DataFrame(data, columns=['Accession','Start','End','Label'])

In [58]:
## query sequences from database, then write sequences into the written fasta file 
# param = {'write_file':True,'filename':'dataset/train_data/gicluster_data.fasta'}                        
# temp = data_df.progress_apply(lambda x: df_to_fasta(x,
#                                           dna_only=True,
#                                           query_db=True,
#                                           **param),
#                         axis=1)

### output: gicluster_data.fasta

## Dataset from RVM

In [59]:
#RVM: https://doi.org/10.1093/bioinformatics/btl369 

filename = 'dataset/prev_studies/RVM/SuppTable1.xlsx'
salmonella = pd.read_excel(filename, sheet_name='Salmonella')
staphy = pd.read_excel(filename, sheet_name='Staphylococcus')
strepto = pd.read_excel(filename, sheet_name='Streptococcus')

salmonella = salmonella.rename(columns={'FROM':'Start', 'TO':'End', 'LABEL': 'ID', 'CLASS': 'Label'})
staphy = staphy.rename(columns={'FROM':'Start', 'TO':'End', 'LABEL': 'ID', 'CLASS': 'Label'})
strepto = strepto.rename(columns={'FROM':'Start', 'TO':'End', 'LABEL': 'ID', 'CLASS': 'Label'})

In [60]:
# extract the identifier (species) from the ID
salmonella['identifier'] = salmonella.progress_apply(lambda x: x['ID'].split('.')[1], axis=1)
staphy['identifier'] = staphy.progress_apply(lambda x: x['ID'].split('.')[1], axis=1)
strepto['identifier'] = strepto.progress_apply(lambda x: x['ID'].split('.')[1], axis=1)

100%|██████████| 107/107 [00:00<00:00, 93595.52it/s]


In [61]:
set(salmonella.progress_apply(lambda x: x['ID'].split('.')[1], axis=1).values)

100%|██████████| 421/421 [00:00<00:00, 127808.48it/s]


{'CT18', 'DT104', 'LT2', 'McC', 'SAR', 'SB', 'SEN', 'SG', 'SL1344', 'TY2'}

In [62]:
#map the strains to their accession number from table 4: https://genome.cshlp.org/content/18/2/331/T5.expansion.html

salmonella_dict = {
    'CT18': 'AL513382',
    'DT104': 'HF937208.1', 
    'LT2': 'AE006468', 
    'McC':'CP000026', 
    'SAR': 'CP000880.1', 
    'SB': 'NC_015761', 
    'SEN': 'CP146375.1', 
    'SG': 'AM933173.1', 
    'SL1344': 'NC_016810.1',
    'TY2': 'AE014613'
}

staphy_dict = {
    'Epid_ATCC': 'AE015929', 
    'Epid_RP62': 'NC_002976.3', 
    'Haem': 'AP006716', 
    'MRSA252': 'BX571856', 
    'Mu50': 'BA000017', 
    'MW2': 'BA000033', 
    'N315': 'BA000018',
    'RF122': ' AJ938182.1', 
    'Sapro': 'AP008934', 
    'USA': 'CP000255'
}

strepto_dict = {
    'agal909': 'NC_007432.1', 
    'agalNEM': 'AL732656', 
    'pneumR6': 'AE007317', 
    'pneumTIGR': 'AE005672', 
    'Pyog10750': 'CP000262',
    'Pyog2096': 'CP000261', 
    'Pyog9429': 'CP000259', 
    'PyogSanger': 'AM295007', 
    'suis': 'NC_012925.1', 
    'therm1066': 'CP000024',
    'therm18311': 'CP000023', 
    'Uberis': 'NC_012004.1'
}

salmonella['Accession'] = salmonella.progress_apply(lambda x: salmonella_dict[x['identifier']], axis=1)
staphy['Accession'] = staphy.progress_apply(lambda x: staphy_dict[x['identifier']], axis=1)
strepto['Accession'] = strepto.progress_apply(lambda x: strepto_dict[x['identifier']], axis=1)

rvm_dataset = pd.concat([salmonella,staphy,strepto])[['Accession','Start','End','Label']]
rvm_dataset['Label'] = rvm_dataset['Label'].astype(str) 


100%|██████████| 107/107 [00:00<00:00, 85386.33it/s]


In [63]:
# param = {'write_file':True,'filename':'dataset/train_data/rvm_data.fasta'}                        
# temp = rvm_dataset.progress_apply(lambda x: df_to_fasta(x,
#                                           dna_only=True,
#                                           query_db=True,
#                                           **param),
#                         axis=1)

### output: rvm_data.fasta

## Summary of Datasets

In [67]:
train_dataset = ['benbow','islandpick','rvm','gicluster']
train_df = pd.DataFrame()
for data_name in train_dataset:
    file = "dataset/train_data/{}_data.fasta".format(data_name)
    df = fasta_to_df(file)
    df = df.assign(data_name=data_name)
    train_df = pd.concat([train_df,df])

test_dataset = ['benbow_test', 'literature']

test_df = pd.DataFrame()
for data_name in test_dataset:
    file = "dataset/test_data/{}_data.fasta".format(data_name)
    df = fasta_to_df(file)
    df = df.assign(data_name=data_name)
    test_df = pd.concat([test_df,df])

all_data_df = pd.concat([train_df, test_df]).drop_duplicates()
all_data_df['Length'] = all_data_df.apply(lambda x: x['End']-x['Start'], axis=1)
all_data_df['Organism'] = all_data_df.apply(lambda x: ' '.join(x['Description'].split(' ')[1:-2]), axis=1)


summary_df = pd.DataFrame()

for data_name in train_dataset+test_dataset:

    cols = ['data_name','train/test','n_organism','n_pos','n_neg']

    #summary of each data set
    data = all_data_df[all_data_df['data_name']==data_name]

    if data_name in train_dataset:
        train_test = 'train'
    elif data_name in test_dataset:
        train_test = 'test'

    row = [data_name,train_test,data['Accession'].unique().size,data['Label'].tolist().count('1'),
    data['Label'].tolist().count('0')]

    summary_df = pd.concat([summary_df,pd.DataFrame([row], columns=cols)])

In [68]:
summary_df

,data_name,train/test,n_organism,n_pos,n_neg
0,benbow,train,167,1742,1393
0,islandpick,train,104,1845,3266
0,rvm,train,32,331,337
0,gicluster,train,9,625,1743
0,benbow_test,test,20,413,153
0,literature,test,6,80,182


In [69]:
# length of sequences in each data set
#statistics of the sequence length of each dataset
all_data_df.groupby(['data_name','Label'])['Length'].agg(['mean', 'std']).reset_index()

,data_name,Label,mean,std
0,benbow,0,14034.557789,8094.962113
1,benbow,1,14725.681401,19772.416757
2,benbow_test,0,12238.124183,4547.176543
3,benbow_test,1,11070.723971,8219.773340
4,gicluster,0,6563.792886,4624.818806
5,gicluster,1,5010.166400,5209.246204
6,islandpick,0,13799.303735,8038.640713
7,islandpick,1,11599.647154,8974.758666
8,literature,0,12042.598901,4022.119405
9,literature,1,38297.575000,24925.249354


## Check Species Diversity in each data set

In [70]:
from scipy.stats import entropy

# Function to calculate Species Richness
def species_richness(species_counts):
    return np.count_nonzero(species_counts)

# Function to calculate Shannon Index (H')
def shannon_index(species_counts):
    proportions = species_counts / np.sum(species_counts)
    return entropy(proportions, base=np.e)

# Function to calculate Simpson's Index (D)
def simpsons_index(species_counts):
    proportions = species_counts / np.sum(species_counts)
    return 1 - np.sum(proportions**2)

In [71]:
all_data_df['genus'] = all_data_df.apply(lambda x: x['Organism'].split(' ')[0], axis=1)
all_data_df['species'] = all_data_df.apply(lambda x: ' '.join(x['Organism'].split(' ')[0:2]), axis=1)

In [72]:
species_count = all_data_df.groupby(['data_name','species']).size().reset_index(name='counts')
species_count['p'] = species_count.groupby('data_name').apply(lambda x: x['counts']/x['counts'].sum()).reset_index(name='p')['p'] 
species_count['ln_p'] = species_count.apply(lambda x: np.log(x['p']), axis=1)
species_count['p*ln_p'] = species_count.apply(lambda x: -1*x['p']*x['ln_p'], axis=1) 

In [73]:
species_diversity = pd.DataFrame()
species_diversity['species_richness'] = species_count.groupby('data_name').apply(lambda x: species_richness(x['counts']))
species_diversity['shannon_index'] = species_count.groupby('data_name').apply(lambda x: shannon_index(x['counts']))
species_diversity['simpsons_index'] = species_count.groupby('data_name').apply(lambda x: simpsons_index(x['counts']))

In [74]:
species_diversity.sort_values(by='species_richness', ascending=False)

,species_richness,shannon_index,simpsons_index
data_name,,,
benbow,79,3.626918,0.964113
islandpick,50,3.523437,0.958348
benbow_test,16,2.277212,0.837112
rvm,12,1.722087,0.725205
gicluster,9,2.141821,0.876815
literature,5,1.366144,0.690257


In [75]:
genus_count = all_data_df.groupby(['data_name','genus']).size().reset_index(name='counts')
genus_count['p'] = genus_count.groupby('data_name').apply(lambda x: x['counts']/x['counts'].sum()).reset_index(name='p')['p'] 
genus_count['ln_p'] = genus_count.apply(lambda x: np.log(x['p']), axis=1)
genus_count['p*ln_p'] = genus_count.apply(lambda x: -1*x['p']*x['ln_p'], axis=1) 

In [76]:
genus_diversity = pd.DataFrame()
genus_diversity['genus_richness'] = genus_count.groupby('data_name').apply(lambda x: species_richness(x['counts']))
genus_diversity['shannon_index'] = genus_count.groupby('data_name').apply(lambda x: shannon_index(x['counts']))
genus_diversity['simpsons_index'] = genus_count.groupby('data_name').apply(lambda x: simpsons_index(x['counts']))

In [77]:
genus_diversity.sort_values(by='genus_richness', ascending=False)

,genus_richness,shannon_index,simpsons_index
data_name,,,
benbow,33,2.605827,0.894077
islandpick,22,2.598021,0.896702
benbow_test,10,1.947907,0.813408
gicluster,9,2.141821,0.876815
literature,5,1.366144,0.690257
rvm,3,0.911817,0.533217
